In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [2]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [3]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [4]:
# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Use 'sigmoid'

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy','Precision','Recall','AUC']
)

In [6]:
batch = 32
classes = 'binary'

train='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/train'
test='C:/Users/Shusmita\Desktop\Oral Cancer/Augmented_Oral_Cancer/test'
val='C:/Users/Shusmita/Desktop/Oral Cancer/Augmented_Oral_Cancer/Validate'

In [7]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validation_datagen= image.ImageDataGenerator()

test_datagen= image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)


validation_generator = validation_datagen.flow_from_directory(
    val,
    target_size = (224,224),
    batch_size = batch,
    shuffle=True,
    class_mode = classes)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)
class_names=validation_generator.class_indices
class_names
len(class_names)

Found 619 images belonging to 2 classes.
Found 130 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


2

In [8]:
# Train the model
epochs = 5  # You can adjust the number of epochs
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/5
20/20 [==============================] - 50s 2s/step - loss: 0.7366 - accuracy: 0.6607 - precision: 0.6583 - recall: 0.6752 - auc: 0.7088 - val_loss: 0.5161 - val_accuracy: 0.7692 - val_precision: 0.8400 - val_recall: 0.6562 - val_auc: 0.8267
Epoch 2/5
20/20 [==============================] - 26s 1s/step - loss: 0.5804 - accuracy: 0.7189 - precision: 0.7231 - recall: 0.7138 - auc: 0.7844 - val_loss: 0.5233 - val_accuracy: 0.7538 - val_precision: 0.9444 - val_recall: 0.5312 - val_auc: 0.8782
Epoch 3/5
20/20 [==============================] - 31s 2s/step - loss: 0.5716 - accuracy: 0.7157 - precision: 0.7077 - recall: 0.7395 - auc: 0.7879 - val_loss: 0.5519 - val_accuracy: 0.7077 - val_precision: 1.0000 - val_recall: 0.4062 - val_auc: 0.8924
Epoch 4/5
20/20 [==============================] - 30s 1s/step - loss: 0.4941 - accuracy: 0.7528 - precision: 0.7633 - recall: 0.7363 - auc: 0.8416 - val_loss: 0.4179 - val_accuracy: 0.8000 - val_precision: 0.8519 - val_recall: 0.7188 - val_

In [9]:
score=model.evaluate(validation_generator)
score

5/5 [==============================] - 4s 774ms/step - loss: 0.4665 - accuracy: 0.7923 - precision: 0.9111 - recall: 0.6406 - auc: 0.8838


[0.46645551919937134,
 0.7923076748847961,
 0.9111111164093018,
 0.640625,
 0.8837594985961914]

In [10]:
score=model.evaluate(test_generator)
score

5/5 [==============================] - 6s 1s/step - loss: 0.4449 - accuracy: 0.7786 - precision: 0.8909 - recall: 0.6622 - auc: 0.9049


[0.44488728046417236,
 0.7785714268684387,
 0.8909090757369995,
 0.662162184715271,
 0.9048935174942017]

In [11]:
score=model.evaluate(train_generator)
score

20/20 [==============================] - 32s 2s/step - loss: 0.4336 - accuracy: 0.8126 - precision: 0.8545 - recall: 0.7556 - auc: 0.8988


[0.4336383044719696,
 0.8126009702682495,
 0.8545454740524292,
 0.7556270360946655,
 0.898813009262085]